In [ ]:
import cv2
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
from torchvision.models import mobilenet_v3_large, vit_b_32
from torch import nn
from pycocotools.coco import COCO
from mean_average_precision import MetricBuilder
import time
from tqdm import tqdm
import json
import os
import time
from retinaface import RetinaFace

IMG_SIZE = 224
NUM_CLASSES = 7
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_dir = "E://Kuliah//UPI//SEMESTER 8//dataset skripsi//dataset ruang kelas final//cek"
output_img_dir = "E://Kuliah//UPI//SEMESTER 8//dataset skripsi//dataset ruang kelas final//hasil2"
os.makedirs(output_img_dir, exist_ok=True)

label_map = {
    0: "Angry",
    1: "Disgust",
    2: "Fear",
    3: "Happy",
    4: "Sad",
    5: "Surprise",
    6: "Neutral"
}

transform = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
    ])

class TokenPooling(nn.Module):
    def __init__(self, keep_tokens: int, use_weighted: bool = True):
        super().__init__()
        self.keep_tokens = keep_tokens
        self.use_weighted = use_weighted

    def forward(self, x: torch.Tensor, significance: torch.Tensor = None) -> torch.Tensor
        B, N_plus_1, D = x.shape
        cls_token, tokens = x[:, :1, :], x[:, 1:, :]  # (B, 1, D), (B, N, D)

        if self.keep_tokens >= tokens.shape[1]:
            return x  # tidak perlu pooling

        if not self.use_weighted:
            significance = torch.ones(tokens.shape[:2], device=x.device)

        # Ambil top-k token berdasarkan skor
        topk_scores, topk_indices = torch.topk(significance, self.keep_tokens, dim=1)  # (B, K)

        # Ambil token berdasarkan indeks top-k
        B_idx = torch.arange(B, device=x.device).unsqueeze(1).expand(-1, self.keep_tokens)  # (B, K)
        pooled_tokens = tokens[B_idx, topk_indices]  # (B, K, D)

        return torch.cat([cls_token, pooled_tokens], dim=1)  # (B, K+1, D)

class MultiheadSelfAttentionBlock(nn.Module):
    def __init__(self,
                 embedding_dim:int=768,
                 num_heads:int=12,
                 attn_dropout:float=0.):
        super().__init__()
        self.multihead_attn = nn.MultiheadAttention(embed_dim=embedding_dim,
                                                    num_heads=num_heads,
                                                    dropout=attn_dropout,
                                                    batch_first=True)
        self.attn_weights = None
    def forward(self, x):
        attn_output, attn_weights = self.multihead_attn(query=x,
                                             key=x,
                                             value=x,
                                             need_weights=True,
                                             average_attn_weights=False)
        self.attn_weights = attn_weights
        return attn_output, attn_weights
    
class MLPBlock(nn.Module):
    def __init__(self,
                 embedding_dim:int=768,
                 mlp_size:int=3072,
                 dropout:float=0.):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(in_features=embedding_dim,
                      out_features=mlp_size),
            nn.GELU(),
            nn.Dropout(p=dropout),
            nn.Linear(in_features=mlp_size,
                      out_features=embedding_dim),
            nn.Dropout(p=dropout)
        )
    def forward(self, x):
        x = self.mlp(x)
        return x

class TransformerEncoderBlock(nn.Module):
    def __init__(self,
                 embedding_dim:int=768,
                 num_heads:int=12,
                 mlp_size:int=3072,
                 mlp_dropout:float=0.,
                 attn_dropout:float=0.):
        super().__init__()
        self.layer_norm1 = nn.LayerNorm(normalized_shape=embedding_dim, eps=1e-6)
        
        self.msa_block = MultiheadSelfAttentionBlock(embedding_dim=embedding_dim,
                                                     num_heads=num_heads,
                                                     attn_dropout=attn_dropout)
        
        self.layer_norm2 = nn.LayerNorm(normalized_shape=embedding_dim, eps=1e-6)
        
        self.mlp_block =  MLPBlock(embedding_dim=embedding_dim,
                                   mlp_size=mlp_size,
                                   dropout=mlp_dropout)
    def forward(self, x):
        x_attn, attn_weights = self.msa_block(self.layer_norm1(x))
        x = x_attn + x
        
        x = self.mlp_block(self.layer_norm2(x)) + x 
        
        return x, attn_weights

class ViTMobilenet(nn.Module):
    def __init__(self,
                 img_size:int=224, # Training resolution from Table 3 in ViT paper
                 in_channels:int=3, # Number of channels in input image
                 patch_size:int=16, # Patch size
                 num_transformer_layers:int=12, # Layers from Table 1 for ViT-Base
                 embedding_dim:int=768, # Hidden size D from Table 1 for ViT-Base
                 mlp_size:int=3072, # MLP size from Table 1 for ViT-Base
                 num_heads:int=12, # Heads from Table 1 for ViT-Base
                 attn_dropout:float=0., # Dropout for attention projection
                 mlp_dropout:float=0., # Dropout for dense/MLP layers 
                 embedding_dropout:float=0., # Dropout for patch and position embeddings
                 num_classes:int=1000): # Default for ImageNet but can customize this
        super().__init__()
         
        assert img_size % 32 == 0, f"Image size must be divisible by 32, image size: {img_size}"
        
        self.mobilenet = mobilenet_v3_large(pretrained=True).features
        
        self.projection = nn.Conv2d(in_channels=960, 
                                    out_channels=embedding_dim,
                                    kernel_size=1)
                 
        self.class_embedding = nn.Parameter(data=torch.randn(1, 1, embedding_dim),
                                            requires_grad=True)

        self.num_patches = (img_size // 32) ** 2  # MobileNet reduces spatial size by 32x
        
        self.position_embedding = nn.Parameter(data=torch.randn(1, self.num_patches+1, embedding_dim),
                                               requires_grad=True)
                
        self.embedding_dropout = nn.Dropout(p=embedding_dropout)
        
        self.transformer_encoder = nn.Sequential(*[TransformerEncoderBlock(embedding_dim=embedding_dim,
                                                                            num_heads=num_heads,
                                                                            mlp_size=mlp_size,
                                                                            mlp_dropout=mlp_dropout) for _ in range(num_transformer_layers)])
        
        self.keep_tokens = [49, 35, 35, 35, 26, 26, 20, 20, 20, 12, 12, 12]
        # Tambahkan TokenPooling per layer (jumlah token disesuaikan)
        self.token_pools = nn.ModuleList([
            TokenPooling(keep_tokens=k, use_weighted=True) for k in self.keep_tokens
        ])
       
        self.norm = nn.LayerNorm(normalized_shape=embedding_dim, eps=1e-6)
        self.head = nn.Linear(in_features=embedding_dim, out_features=num_classes)
    
    def forward(self, pixel_values, labels=None):
        
        batch_size = pixel_values.shape[0]

        # Extract features using MobileNet
        features = self.mobilenet(pixel_values)  # Output shape: (batch_size, 1280, H', W')
        features = self.projection(features)  # Project to embedding_dim: (batch_size, embedding_dim, H', W')

        # Flatten the feature maps into a sequence of tokens
        features = features.flatten(2).transpose(1, 2)  # Shape: (batch_size, num_patches, embedding_dim)
        
        class_token = self.class_embedding.expand(batch_size, -1, -1)

        x = torch.cat((class_token, features), dim=1)  # Shape: (batch_size, num_patches + 1, embedding_dim)

        x = x + self.position_embedding

        x = self.embedding_dropout(x)
        
        significance_scores = []

        for i, block in enumerate(self.transformer_encoder):
            x, attn_weights = block(x)
            
            # Hitung significance score: total attention yang diterima setiap token
            score = attn_weights.sum(dim=1).sum(dim=1)[:, 1:]  # shape: (B, N-1)
            
            significance_scores.append(score)
            
            if self.token_pools[i].keep_tokens > 0:
                x = self.token_pools[i](x, significance=score)
            else:
                x = x[:, :1, :]  # hanya CLS token

        x = self.norm(x)
        
        cls_token_final = x[:, 0]

        logits = self.head(cls_token_final)

        return logits
    
best_model = ViTMobilenet(num_classes=7, 
                     in_channels=3,  
                     num_heads=12, 
                     embedding_dim=768, 
                     num_transformer_layers=12,
                     mlp_size=3072)
checkpoint = torch.load("E://Kuliah//UPI//SEMESTER 8//coba coba//hybrid_mobilenet_vit_merge_all_best.pt")
best_model.load_state_dict(checkpoint["model_state_dict"])
best_model.eval().to(DEVICE)

# best_model = vit_b_32(pretrained=False)
# best_model.heads.head = nn.Linear(best_model.heads.head.in_features, 7)
# best_model = best_model.to(DEVICE)

# checkpoint = torch.load("E://Kuliah//UPI//SEMESTER 8//coba coba//vit_pretrained_best.pt")
# best_model.load_state_dict(checkpoint["model_state_dict"])
# best_model.eval()

# best_model = mobilenet_v3_large(pretrained=False)
# best_model.classifier[3] = nn.Linear(best_model.classifier[3].in_features, 7)
# best_model = best_model.to(DEVICE)

# checkpoint = torch.load("E://Kuliah//UPI//SEMESTER 8//coba coba//mobilenetv3_pretrained_best.pt")
# best_model.load_state_dict(checkpoint["model_state_dict"])
# best_model.eval()

# Warm-up
# model.eval()
dummy_input = torch.randn(1, 3, 224, 224).to(DEVICE)
with torch.no_grad():
    for _ in range(5):
        _ = best_model(dummy_input)


image_extensions = ('.jpg', '.jpeg', '.png')

prediction_output = []
frame_count = 0

total_detection_time = 0
total_crop_time = 0
total_transform_time = 0
total_classif_time = 0
total_draw_time = 0

start_time = time.time()

for filename in tqdm(sorted(os.listdir(image_dir))):
    if not filename.lower().endswith(image_extensions):
        continue

    img_path = os.path.join(image_dir, filename)
    img = cv2.imread(img_path)
#     img = cv2.resize(img, (640, 480))

    if img is None:
        continue


    # Deteksi wajah
    t1 = time.time()
    faces = RetinaFace.detect_faces(img)
    t2 = time.time()
    total_detection_time += (t2 - t1)
    face_tensors = []
    boxes = []

    for face_key, face_data in faces.items():
        facial_area = face_data["facial_area"]
        x1, y1, x2, y2 = facial_area
        
        t3 = time.time()
        face_crop = img[y1:y2, x1:x2]
        t4 = time.time()
        total_crop_time += (t4 - t3)
        
        t5 = time.time()
        face_img = Image.fromarray(cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB))
        try:
            tensor = transform(face_img)
            face_tensors.append(tensor)
            t6 = time.time()
            total_transform_time += (t6 - t5)
            boxes.append((x1, y1, x2, y2))
        except:
            continue
    
    if len(face_tensors) > 0:
        batch_tensor = torch.stack(face_tensors).to(DEVICE)
        with torch.no_grad():
            t7 = time.time()
            logits = best_model(batch_tensor)
            t8 = time.time()
            total_classif_time += (t8 - t7)
            probs = torch.softmax(logits, dim=1)
            class_ids = torch.argmax(probs, dim=1).cpu().numpy()
            scores = torch.max(probs, dim=1).values.cpu().numpy()

        for i, (x1, y1, x2, y2) in enumerate(boxes):
            t9 = time.time()
            label = label_map.get(class_ids[i], "Unknown")
            score = scores[i]
            label_text = f"{label}: {score:.2f}"

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, label_text, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
            t10 = time.time()
            total_draw_time += (t10 - t9)

    # Save the output image
    output_path = os.path.join(output_img_dir, filename)
    cv2.imwrite(output_path, img)
    frame_count += 1

end_time = time.time()
# Calculate FPS
total_time = end_time - start_time
fps = frame_count / total_time
avg_time_per_inference_sec = total_time / frame_count
avg_time_per_inference_ms = avg_time_per_inference_sec * 1000

print(f"Processed {frame_count} images in {total_time:.2f} seconds")
print(f"FPS: {fps:.2f}")
print(f"Waktu rata-rata inferensi: {avg_time_per_inference_ms:.3f} ms")

print(f"Waktu total deteksi: {total_detection_time:.2f} seconds")
print(f"Waktu total crop: {total_crop_time:.2f} seconds")
print(f"Waktu total transform: {total_transform_time:.2f} seconds")
print(f"Waktu total klasifikasi: {total_classif_time:.2f} seconds")
print(f"Waktu total draw: {total_draw_time:.2f} seconds")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:01<00:00,  1.81s/it]

Processed 100 images in 181.15 seconds
FPS: 0.55
Waktu rata-rata inferensi: 1811.512 ms
Waktu total deteksi: 137.99 seconds
Waktu total crop: 0.06 seconds
Waktu total transform: 5.49 seconds
Waktu total klasifikasi: 3.38 seconds
Waktu total draw: 0.30 seconds
